In [3]:
from dataclasses import dataclass
from random import Random
from typing import List, Tuple, Dict


@dataclass(frozen=True)
class ResultCell:
    i: int
    k: int


@dataclass
class Val:
    tag: str
    j: int
    value: float


def mapper_a(A: List[List[float]], cols_b: int) -> List[Tuple[ResultCell, Val]]:
    rows = len(A)
    cols = len(A[0])
    result = []
    for i in range(rows):
        for j in range(cols):
            cell_value = A[i][j]
            for k in range(cols_b):
                result.append((ResultCell(i, k), Val('A', j, cell_value)))
    return result


def mapper_b(B: List[List[float]], rows_a: int) -> List[Tuple[ResultCell, Val]]:
    rows = len(B)
    cols = len(B[0])
    result = []
    for j in range(rows):
        for k in range(cols):
            cell_value = B[j][k]
            for i in range(rows_a):
                result.append((ResultCell(i, k), Val('B', j, cell_value)))
    return result


def shuffler(mapped: List[Tuple[ResultCell, Val]]) -> Dict[ResultCell, List[Val]]:
    sorted_mapped = sorted(mapped, key=lambda x: (x[0].i, x[0].k, x[1].tag, x[1].j))
    result: Dict[ResultCell, List[Val]] = {}
    for key, value in sorted_mapped:
        if key not in result:
            result[key] = []
        result[key].append(value)
    return result


def reducer(grouped: Dict[ResultCell, List[Val]], rows_c: int, cols_c: int) -> List[List[float]]:
    C = [[0.0] * cols_c for _ in range(rows_c)]
    
    for result_cell, values in grouped.items():
        row_values_a: Dict[int, float] = {}
        column_values_b: Dict[int, float] = {}
        
        for val in values:
            if val.tag == 'A':
                row_values_a[val.j] = val.value
            elif val.tag == 'B':
                column_values_b[val.j] = val.value
        
        total = 0.0
        for j, value_a in row_values_a.items():
            if j in column_values_b:
                total += value_a * column_values_b[j]
        
        C[result_cell.i][result_cell.k] = total
    
    return C


def multiply_map_reduce(A: List[List[float]], B: List[List[float]]) -> List[List[float]]:
    rows_a = len(A)
    cols_a = len(A[0])
    rows_b = len(B)
    cols_b = len(B[0])
    
    if cols_a != rows_b:
        raise ValueError(
            f"Размерности матриц не позволяют выполнить операцию умножения:\n\tA - {rows_a}x{cols_a}, B - {rows_b}x{cols_b}"
        )
    
    mapped_a = mapper_a(A, cols_b)
    mapped_b = mapper_b(B, rows_a)
    
    grouped = shuffler(mapped_a + mapped_b)
    
    return reducer(grouped, rows_a, cols_b)

def generate_matrix(rows: int, cols: int, min: float=0.0, max: float=1.0, seed: int=42):
    rand = Random(seed)
    return [
        [rand.uniform(min, max) for _ in range(cols)]
        for _ in range(rows)
    ]

def print_matrix(matrix: List[List[float]], title:str=""):
    if len(title) > 0:
        print(title)
    for row in matrix:
        print(", ".join(f"{x}" for x in row))

if __name__ == "__main__":
    Ar, Ac, Bc = 150, 200, 150

    A = generate_matrix(rows=Ar,cols=Ac,min=0,max=10,seed=92133)
    B = generate_matrix(rows=Ac,cols=Bc,min=-2,max=2,seed=23921)
    print_matrix(A, "Матрица A:")
    print_matrix(B, "\n\nМатрица B:")

    C = multiply_map_reduce(A, B)
    print_matrix(C, "\n\nРезультат:")

Матрица A:
7.833311353493986, 5.567160810357064, 8.718665191946837, 4.8661310503661, 1.6644552430202364, 6.862897023349188, 3.135384650781372, 9.7802987527409, 7.491791397306678, 0.8348236236855588, 2.1929397864276234, 6.055114508616459, 5.537471406464497, 8.853789235263449, 7.768152250008202, 4.164112994850359, 4.95627582083359, 4.865662580897886, 0.23462352164423717, 6.012058382976928, 1.3754287929974118, 6.112116463127434, 6.045382101434558, 0.9153064299884583, 2.090545773661694, 3.905417689868498, 9.623495860787372, 1.5548295986226368, 6.189996985762548, 3.0126341107666867, 9.19449962397174, 9.820562450867117, 1.9367144602555586, 0.5537689821207603, 9.099652856886427, 8.120967512734756, 8.687790832193548, 9.245277509109666, 7.641156361087342, 6.655072426902462, 8.018384264269486, 2.7619411062920154, 5.716559507292368, 6.507953487634421, 4.859278823526396, 6.385693421192787, 5.747620119705634, 2.232378067797347, 1.534166138361096, 7.955384227355763, 9.596896284564165, 6.030851263601